In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML

from cytoolz import concatv

# NLTK for NLP utils and corpora
import nltk,pprint
from nltk import word_tokenize
nltk.download('treebank')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))


import pandas as pd
import pickle
import string
import copy

[nltk_data] Downloading package treebank to
[nltk_data]     /home/huyue012/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /home/huyue012/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/huyue012/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/huyue012/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def sent_tokenize_rm_punct (blob):
    bexp = ''    
    blob.replace('\n','')
    #blob.replace('</p>','  ')
    #print(type(blob))

    for i, char in (enumerate(blob)):
        #print(i,char)
        next_cap = False
        prev_lower = False
        next_space = False
        
        if char in string.punctuation :
            #print("found punctuation:", i, char, blob[i+1])
           
            if char in '.?!' :
                if i+1 >= len(blob):
                    next_cap = False
                else:
                    next_cap = blob[i+1].isupper()
                    next_space = blob[i+1].isspace()                    

                if i-1 <0:
                    prev_lower = False
                else:
                    prev_lower = blob[i-1].islower()

                if (next_cap and prev_lower) or next_space: 
                    # if the char before "." is lower case, but the one immediately follow the "." is Uppercase, then this is a paragraph end (caused by removing <p></p> from the xml file)
                    bexp = bexp + ' </s> '
                else:
                    #if this is not end of paragraph, then 
                    #bexp = bexp + char
                    pass
            else :
                bexp = bexp + ' '
        elif char == '\n':
            i += 1                    
        else :
            if char.isnumeric() : 
                bexp = bexp + 'N'
            else :
                bexp = bexp + char.lower()
    
    #return(sent_tokenize(bexp))
    return(bexp)




In [6]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
 
data = "All work and no play makes jack dull boy. All work and no play makes jack a dull boy."
stopWords = set(stopwords.words('english'))
words = word_tokenize(data)
wordsFiltered = []
 
for w in words:
    if w not in stopWords:
        wordsFiltered.append(w)

print(wordsFiltered)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/huyue012/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['All', 'work', 'play', 'makes', 'jack', 'dull', 'boy', '.', 'All', 'work', 'play', 'makes', 'jack', 'dull', 'boy', '.']


In [5]:
# # top N vocab?
# def create_lookup_tables(text, V=10000):
#     # make a list of unique words
#     CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 , '<p>': 4}
#     print("length of create_lookup_table input:", len(text))
#     print("type of create_lookup_table input:", type(text))
    
#     words = text.split() #??
#     unigram_counts = Counter()
#     prev_word = None
#     for w in words:
#         unigram_counts[w] += 1
#         prev_word = word

#     # Leave space for codes above
#     top_counts = unigram_counts.most_common(V - len(CODES))

# #     vocab = set(words)
#     vocab = [w for w,c in top_counts]

#     # (1)
#     # starts with the special tokens
#     vocab_to_int = copy.copy(CODES)

#     # the index (v_i) will starts from 4 (the 2nd arg in enumerate() specifies the starting index)
#     # since vocab_to_int already contains special tokens
#     for v_i, v in enumerate(vocab, len(CODES)):
#         vocab_to_int[v] = v_i
#         #print(v_i, v)
#     # (2)
#     int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}
# #     print(len(vocab_to_ind), vocab_to_int[:6])
    
#     return vocab_to_int, int_to_vocab


In [3]:
def create_lookup_tables(text):
    # make a list of unique words
    CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 , '<p>': 4}
    print("length of create_lookup_table input:", len(text))
    print("type of create_lookup_table input:", type(text))
    
    vocab = set(text.split())
#     vocab = set(''.join(list(body_text)).split() + text.split())
    
    # remove stop words
    stopWords = set(stopwords.words('english'))
    vocab_filtered = [w for w in vocab if w not in stopWords]
    
    # (1)
    # starts with the special tokens
    vocab_to_int = copy.copy(CODES)

    # the index (v_i) will starts from 4 (the 2nd arg in enumerate() specifies the starting index)
    # since vocab_to_int already contains special tokens
    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i
        #print(v_i, v)
    # (2)
    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
        1st, 2nd args: raw string text to be converted
        3rd, 4th args: lookup tables for 1st and 2nd args respectively
    
        return: A tuple of lists (source_id_text, target_id_text) converted
    """
    # empty list of converted sentences
    source_text_id = []
    target_text_id = []
    
    # make a list of sentences (extraction)
    #source_sentences = source_text.split("\n")
    #target_sentences = target_text.split("\n")
    source_sentences = source_text
    target_sentences = target_text

    
    max_source_sentence_length = max([len(sentence.split(" ")) for sentence in source_sentences])
    max_target_sentence_length = max([len(sentence.split(" ")) for sentence in target_sentences])
    
    # iterating through each sentences (# of sentences in source&target is the same)
    for i in range(len(source_sentences)):
        # extract sentences one by one
        source_sentence = source_sentences[i]
        target_sentence = target_sentences[i]
        
        # make a list of tokens/words (extraction) from the chosen sentence
        source_tokens = source_sentence.split(" ")
        target_tokens = target_sentence.split(" ")
        
        # empty list of converted words to index in the chosen sentence
        source_token_id = []
        target_token_id = []
        
        for index, token in enumerate(source_tokens):
            if (token != ""):
                source_token_id.append(source_vocab_to_int[token])
        
        for index, token in enumerate(target_tokens):
            if (token != ""):
                target_token_id.append(target_vocab_to_int[token])
                
        # put <EOS> token at the end of the chosen target sentence
        # this token suggests when to stop creating a sequence
        target_token_id.append(target_vocab_to_int['<EOS>'])
            
        # add each converted sentences in the final list
        source_text_id.append(source_token_id)
        target_text_id.append(target_token_id)
    
    return source_text_id, target_text_id

In [5]:
import pickle
def preprocess_and_save_data(source_text, target_text, text_to_ids):
    # Preprocess
    

    # create lookup tables for English and French data
    #source_text is a Pandas dataframe series, so is target_text
    #to create lookup_tables, we will join all the lines together and send as text. 
    source_vocab_to_int, source_int_to_vocab = create_lookup_tables(''.join(list(source_text)))
    target_vocab_to_int, target_int_to_vocab = create_lookup_tables(''.join(list(target_text)))
    

    # create list of sentences whose words are represented in index
    source_text, target_text = text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int)
    
    print(type(source_text), len(source_text))
    print(source_text[10:12])
    # Save data for later use
    pickle.dump((
        (source_text, target_text),
        (source_vocab_to_int, target_vocab_to_int),
        (source_int_to_vocab, target_int_to_vocab)), open('preprocess.p', 'wb'))


def load_preprocess():
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)    

## checkpoint for procedures used to perform pre-processing

In [6]:
import pandas as pd
import pickle
import string
#parameter setting
work_dir = "working_dir/"
#read file into Pandas Dataframe.
#each line is one record, it contains "original file name(including y/m/d)","TITLE","ABSTRACT","LEAD_PARAGRAPH","FULL_TEXT"
nyt_data = pd.read_csv('abstract_nyt_structured_data_1000.csv', 
                       delimiter=',', index_col=0, header=None, quotechar='"', quoting=1, 
                       skipinitialspace=True, engine='c')

#We will use LEAD_PARAGRAPH as source for now
#And ABSTRACT as target. 
# Remove punctuation, change digits to NNN, and to lower case.     
source_text=nyt_data[3].apply(sent_tokenize_rm_punct)
target_text=nyt_data[2].apply(sent_tokenize_rm_punct)
body_text = nyt_data[4].apply(sent_tokenize_rm_punct)
print(type(source_text),source_text[0:5])
print(type(target_text),target_text[0:5])


<class 'pandas.core.series.Series'> 0
/1988/03/23/0129960.xml     new york city s ambitious effort to give physi...
/1988/03/23/0129961.xml     after revealing on national television that he...
/1988/03/23/0129962.xml     while standing  smooth down as much fabric as ...
/1996/06/30/0861580.xml     an imaginative chef can spice up assorted left...
/1996/06/30/0861588.xml     the nutty professoreddie murphy  jada pinkett ...
Name: 3, dtype: object
<class 'pandas.core.series.Series'> 0
/1988/03/23/0129960.xml      new york city dept of health and mental hygie...
/1988/03/23/0129961.xml      gov james e mcgreevey  whose insistence on st...
/1988/03/23/0129962.xml      women wearing miniskirts describe how to sit ...
/1996/06/30/0861580.xml      anna kisselgoff reviews american ballet theat...
/1996/06/30/0861588.xml      laurel graeber reviews the nutty professor as...
Name: 2, dtype: object


In [28]:
# doc, head, abstract, lead paragraph, full
nyt_data.head(2)

,1,2,3,4
0,,,,
/1988/03/23/0129960.xml,"For Health Survey, Many Offer More Excuses Tha...",New York City Dept of Health and Mental Hygie...,New York City's ambitious effort to give physi...,New York City's ambitious effort to give physi...
/1988/03/23/0129961.xml,McGreevey Seems Set To Exit On His Terms,"Gov James E McGreevey, whose insistence on st...",After revealing on national television that he...,After revealing on national television that he...


In [23]:
a = nyt_data[3][1]
print(a)
sent_tokenize_rm_punct(a)

After revealing on national television that he is gay, seeing his name become the punch line of countless late-night comedy gags, and fending off a coup attempt from within his own party, Gov. James E. McGreevey needs only to remain in office until midnight Friday to outlast the state deadline that would automatically force a special election to choose his successor.Mr. McGreevey announced on Aug. 12 that he was stepping down because he had had an extramarital affair with a man he did not identify, but his insistence on remaining in office until Nov. 15 set off political, legal and public relations challenges. 


'after revealing on national television that he is gay  seeing his name become the punch line of countless late night comedy gags  and fending off a coup attempt from within his own party  gov </s>  james e </s>  mcgreevey needs only to remain in office until midnight friday to outlast the state deadline that would automatically force a special election to choose his successor </s> mr </s>  mcgreevey announced on aug </s>  NN that he was stepping down because he had had an extramarital affair with a man he did not identify  but his insistence on remaining in office until nov </s>  NN set off political  legal and public relations challenges </s>  '

In [7]:
print(len(nyt_data),len(source_text))
work_dir = "working_dir/"
work_dir2 = "working_dir2/"
# save_path = work_dir2


print(string.punctuation)

1000 1000
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [8]:
preprocess_and_save_data(source_text, target_text, text_to_ids)

length of create_lookup_table input: 724292
type of create_lookup_table input: <class 'str'>
length of create_lookup_table input: 221117
type of create_lookup_table input: <class 'str'>
<class 'list'> 1000
[[11732, 6583, 481, 1337, 13167, 9850, 3401, 247, 14473, 13654, 13066, 6871, 10166, 14473, 2240, 14452, 2030, 3175, 10166, 7633, 11514, 9850, 5869, 1934, 2905, 14136, 3790, 11920, 10166, 6394, 136, 6583, 573, 8323, 6305, 14239, 12184, 1532, 7267, 14703, 8519, 1022, 14435, 3028, 11514, 12813, 13534, 2119, 8094, 13087, 630, 1495, 13653, 14985, 13534], [320, 6642, 5023, 3153, 2772, 8925, 11732, 5841, 13534, 12083, 3283, 3071, 10127, 2913, 13066, 6583, 6891, 501, 5822, 3028, 5678, 957, 1003, 8018, 2199, 14498, 15442, 11338, 4547, 12660, 14136, 12668, 1910, 12668, 10557, 13117, 327, 10166, 6583, 8653, 9380, 9569, 680, 5841, 10166, 9678, 11206, 13534]]


In [9]:
def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len


In [10]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [11]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state

In [12]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [13]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [14]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            cells, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            cells, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [15]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output

In [21]:
display_step = 10

epochs = 25
batch_size = 50

rnn_size = 200
num_layers = 3

encoding_embedding_size = 200
decoding_embedding_size = 200

learning_rate = 0.01
keep_probability = 0.5
save_path = work_dir

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()

# max_target_sentence_length = max([len(sentence) for sentence in source_int_text])
max_target_sentence_length = min(150,max([len(sentence) for sentence in source_int_text]))

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    #lr, keep_prob = hyperparam_inputs()
    

    lr = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
  

    train_logits, inference_logits = seq2seq_model(input_data, #tf.reverse(input_data, [-1])
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
 
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

## Train

In [22]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
#     max_sentence = max([len(sentence) for sentence in sentence_batch])
    max_sentence = min(150, max([len(sentence) for sentence in sentence_batch]))
    return [sentence[:150] + [pad_int] * (max_sentence - len(sentence[:150])) for sentence in sentence_batch]
#     return [sentence for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))
        

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths
        
        
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                 valid_target,
                 batch_size,
                 source_vocab_to_int['<PAD>'],
                 target_vocab_to_int['<PAD>']))                                                                               

In [23]:
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1})

                train_acc = get_accuracy(target_batch, batch_train_logits)
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')        

Epoch   0 Batch   10/20 - Train Accuracy: 0.6600, Validation Accuracy: 0.7028, Loss: 3.3658
Epoch   1 Batch   10/20 - Train Accuracy: 0.6602, Validation Accuracy: 0.7029, Loss: 3.0010
Epoch   2 Batch   10/20 - Train Accuracy: 0.6605, Validation Accuracy: 0.7035, Loss: 3.0845
Epoch   3 Batch   10/20 - Train Accuracy: 0.6627, Validation Accuracy: 0.7033, Loss: 2.9782
Epoch   4 Batch   10/20 - Train Accuracy: 0.6632, Validation Accuracy: 0.7044, Loss: 2.9363
Epoch   5 Batch   10/20 - Train Accuracy: 0.6636, Validation Accuracy: 0.7048, Loss: 2.9026
Epoch   6 Batch   10/20 - Train Accuracy: 0.6634, Validation Accuracy: 0.7051, Loss: 2.8894
Epoch   7 Batch   10/20 - Train Accuracy: 0.6634, Validation Accuracy: 0.7039, Loss: 2.7879
Epoch   8 Batch   10/20 - Train Accuracy: 0.6632, Validation Accuracy: 0.7057, Loss: 2.7778
Epoch   9 Batch   10/20 - Train Accuracy: 0.6657, Validation Accuracy: 0.7047, Loss: 2.7145
Epoch  10 Batch   10/20 - Train Accuracy: 0.6641, Validation Accuracy: 0.7043, L

In [23]:
# train_source = source_int_text[batch_size:] 
print(len(source_int_text), len(train_source))
print(source_int_text[:2])

1000 950
[[9245, 7334, 23363, 9934, 11280, 22635, 29948, 28905, 30380, 32974, 28803, 5473, 28589, 29948, 27880, 17723, 26196, 20130, 27517, 15987, 22843, 6366, 31669, 15395, 7935, 29097, 29948, 10882, 32431, 13127, 15442, 35895, 12648, 23363, 4238, 35602, 28410, 29948, 18593, 3467, 33243, 29948, 2589, 25364, 5208, 13125, 12164, 3467, 23363, 9934, 4238, 36253, 7077, 33964, 7935, 25364, 11106, 25214, 1759, 36709, 24181, 12648, 31426, 24129, 6366, 3467, 22635, 18758, 30208, 13845, 7077, 9245, 19426, 3624, 13127, 5943, 17283, 31269, 30907, 4906, 12173, 4906, 23155, 28803, 10933, 4906, 20877, 29948, 3739, 12648], [34846, 2835, 1383, 36415, 20045, 7077, 8806, 10286, 1093, 23971, 1010, 32747, 3319, 3467, 12124, 3186, 12164, 25592, 31146, 28791, 10180, 9424, 28803, 30880, 17283, 25364, 3557, 18554, 1759, 36707, 1010, 16315, 21153, 30855, 12648, 35543, 12199, 12648, 18040, 9842, 18365, 29948, 8103, 13127, 17576, 26052, 15942, 28004, 29948, 36330, 3467, 35130, 27795, 7077, 7657, 11584, 8678, 253

In [34]:
print(len(valid_sources_batch))
valid_sources_batch[:2]

50


array([[ 5254,  8036, 14555, 12764,  6366,  7589,  4914, 13653,  6798,
         8726,  5355,  5213,  4815,  4914, 12889,  8771, 15373,  1718,
        14997,  2770, 12947,  6483, 11585,  6119, 10662,   373,  4914,
          451,  3907,    46,  7382, 13938,   885, 14555, 10138, 10245,
        15231,  4914,  1833,  3710,  3543,  4914, 11904,  4028, 13370,
        12347,  3658,  3710, 14555, 12764, 10138,  6205,  7871, 13084,
        10662,  4028,  5004, 12933,  2954,  8870, 14413,   885,  8849,
        15431,  6483,  3710,  7589,  9514, 13098,  1925,  7871,  5254,
         6002,  4775,    46,  7320,  5857, 13282,   945, 15257,  9548,
        15257,  7230,  5355, 14629, 15257,    56,  4914,  9411,   885,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

## inference

In [27]:
def sentence_to_seq(sentence, vocab_to_int):
    results = []
    for word in sentence.split(" "):
        if word in vocab_to_int:
            results.append(vocab_to_int[word])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results
_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()

ex0 = 'he saw a old yellow truck .'
ex1 = nyt_data[3][2]
ex2 = ' '.join([source_int_to_vocab[i] for i in source_int_text[1]])
translate_sentence = ex1

translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(save_path + '.meta')
    loader.restore(sess, save_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from working_dir/
Input
  Word Ids:      [2, 2, 5152, 15397, 12668, 1233, 7052, 12668, 4900, 7751, 10657, 2, 2, 11390, 6583, 2, 9763, 10166, 2, 2, 2, 9452, 13712, 10114, 2, 15053, 2724, 9380, 6583, 5664, 1933, 3028, 15254, 10574, 8314, 2, 4578, 11359, 8314, 2, 12668, 5394, 6968, 11732, 7835, 3071, 3158, 2913, 7751, 3039, 6583, 2267, 10166, 6515, 5394, 320, 2, 2]
  English Words: ['<UNK>', '<UNK>', 'smooth', 'down', 'as', 'much', 'fabric', 'as', 'possible', 'with', 'both', '<UNK>', '<UNK>', 'cross', 'the', '<UNK>', 'bend', 'and', '<UNK>', '<UNK>', '<UNK>', 'place', 'purse', 'over', '<UNK>', 'peculiar', 'choreography', 'is', 'the', 'daily', 'routine', 'for', 'women', 'wearing', 'miniskirts', '<UNK>', 'extremely', 'short', 'miniskirts', '<UNK>', 'as', 'they', 'try', 'to', 'grab', 'a', 'subway', 'seat', 'with', 'all', 'the', 'grace', 'and', 'modesty', 'they', 'can', '<UNK>', '<UNK>']

Prediction
  Word Ids:      [1835, 4919, 5283, 2347, 499, 4919, 7664,

In [30]:
print("Source: ",nyt_data[3][2])
print("\nTarget: ",nyt_data[2][2])
print("\nPrediction: ", " ".join([target_int_to_vocab[i] for i in translate_logits]))

Source:  While standing, smooth down as much fabric as possible with both hands. Then cross the knees, bend and sit. Optional precaution: place purse over lap.This peculiar choreography is the daily routine for women wearing miniskirts -- extremely short miniskirts -- as they try to grab a subway seat with all the grace and modesty they can muster. 

Target:   Women wearing miniskirts describe how to sit on subway seat with modicum of grace and modesty; photos (M)  

Prediction:  new york reviews to sellers of new and long and long and long and long and long and long and NNN and long and NNN and long and NNN and long and NNN and long and NNN and NNN and NNN and NNN and NNN and NNN and NNN and NNN and NNN and NNN <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

In [33]:
print("Full article: ", nyt_data[4][2])

Full article:  While standing, smooth down as much fabric as possible with both hands. Then cross the knees, bend and sit. Optional precaution: place purse over lap.This peculiar choreography is the daily routine for women wearing miniskirts -- extremely short miniskirts -- as they try to grab a subway seat with all the grace and modesty they can muster.The city can often be an obstacle course for fashionistas, with grates that threaten to trip stilettos at every corner and humidity that can turn any fabric into virtual Saran Wrap.But the chic cannot be ruled by fear. So when hemlines that fall just centimeters below one's bottom became all the rage this summer, young women who dared found a way.The goal is to avoid having your bottom ''touch too much of the seat,'' said Jessica Oser, 24, who sported a lavender polo shirt and khaki mini as she browsed at the Union Square farmers' market this week.Ms. Oser, a student at the Benjamin N. Cardozo School of Law, never wears a miniskirt when

In [44]:
' '.join([source_int_to_vocab[i] for i in source_int_text[1]])

'after revealing on national television that he is gay seeing his name become the punch line of countless late night comedy gags and fending off a coup attempt from within his own party gov </s> james e </s> mcgreevey needs only to remain in office until midnight friday to outlast the state deadline that would automatically force a special election to choose his successor </s> mr </s> mcgreevey announced on aug </s> NN that he was stepping down because he had had an extramarital affair with a man he did not identify but his insistence on remaining in office until nov </s> NN set off political legal and public relations challenges </s>'

In [28]:
nyt_data = pd.read_csv('data/abstract_nyt_structured_data.csv', 
                       delimiter=',', index_col=0, header=None, quotechar='"', quoting=1, 
                       skipinitialspace=True, engine='c')

ParserError: Error tokenizing data. C error: EOF inside string starting at line 66036